In [14]:
import json
import pandas as pd
import pymysql

# Cargar el archivo JSON
with open('meteorología_2023.json') as f:
    data = json.load(f)

# Aplanar la estructura del JSON (con manejo de valores faltantes)
flattened_data = []
for date, measurements in data['pollutionMeasurements']['date'].items():
    for station, values in measurements.items():
        flattened_data.append([
            date, 
            station, 
            values.get('RH', None),  # Permitir valores nulos si no existe 'RH'
            values.get('TMP', None), 
            values.get('WDR', None), 
            values.get('WSP', None), 
            values.get('PBa', None)
        ])

# Crear DataFrame con nombres de columnas explícitos
df = pd.DataFrame(flattened_data, columns=['Fecha y Hora', 'Estación', 'RH', 'TMP', 'WDR', 'WSP', 'PBa'])

# Imprimir las primeras filas del DataFrame para verificar su estructura
print(df.head())

# Corregir la hora "24:00" (si existe) a "00:00"
df['Fecha y Hora'] = df['Fecha y Hora'].astype(str).str.replace(' 24:00', ' 00:00', regex=False)

# Convertir la columna 'Fecha y Hora' a datetime
df['Fecha y Hora'] = pd.to_datetime(df['Fecha y Hora'], format='%Y-%m-%d %H:%M')

# Conectar a MySQL (agregando el parámetro charset)
connection = pymysql.connect(
    host='34.72.69.10',
    user='bd_conagua',
    password='HkW0eiFxfT5iUv31',
    database='conagua',
    port=3306,
    charset='utf8mb4'  # Codificación UTF-8 para caracteres especiales
)

cursor = connection.cursor()

# Crear la tabla si no existe 
create_table_query = """
CREATE TABLE IF NOT EXISTS mediciones (
    `Fecha y Hora` DATETIME,
    `Estación` VARCHAR(255),  
    `RH` DECIMAL(5,2),   
    `TMP` DECIMAL(5,2),
    `WDR` DECIMAL(5,2),
    `WSP` DECIMAL(5,2),
    `PBa` DECIMAL(5,2)
)
"""
cursor.execute(create_table_query)

# Insertar los datos en la tabla
insert_query = """
INSERT INTO mediciones (`Fecha y Hora`, `Estación`, `RH`, `TMP`, `WDR`, `WSP`, `PBa`)
VALUES (%s, %s, %s, %s, %s, %s, %s)
"""

# Iterar sobre las filas del DataFrame y preparar los datos para la inserción
data_to_insert = []
for _, row in df.iterrows():
    data_to_insert.append((
        row['Fecha y Hora'],
        row['Estación'],
        row['RH'],
        row['TMP'],
        row['WDR'],
        row['WSP'],
        row['PBa']
    ))

cursor.executemany(insert_query, data_to_insert)

connection.commit()
cursor.close()
connection.close()

print("Datos cargados correctamente en la base de datos.")








       Fecha y Hora Estación    RH   TMP   WDR   WSP   PBa
0  2023-01-01 01:00       RH  None  None  None  None  None
1  2023-01-01 01:00      TMP  None  None  None  None  None
2  2023-01-01 01:00      WDR  None  None  None  None  None
3  2023-01-01 01:00      WSP  None  None  None  None  None
4  2023-01-01 01:00      PBa  None  None  None  None  None
Datos cargados correctamente en la base de datos.


In [ ]:
pip install tabulate


In [12]:
pip install mysql-connector-python


   ---------------------------------------- 0.0/14.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.5 MB 330.3 kB/s eta 0:00:44
   ---------------------------------------- 0.1/14.5 MB 825.8 kB/s eta 0:00:18
    --------------------------------------- 0.2/14.5 MB 1.8 MB/s eta 0:00:08
   - -------------------------------------- 0.5/14.5 MB 3.2 MB/s eta 0:00:05
   --- ------------------------------------ 1.3/14.5 MB 6.5 MB/s eta 0:00:03
   --- ------------------------------------ 1.4/14.5 MB 6.5 MB/s eta 0:00:03
   ---- ----------------------------------- 1.7/14.5 MB 5.7 MB/s eta 0:00:03
   ---- ----------------------------------- 1.7/14.5 MB 5.7 MB/s eta 0:00:03
   ----- ---------------------------------- 2.0/14.5 MB 5.7 MB/s eta 0:00:03
   -------- ------------------------------- 2.9/14.5 MB 6.7 MB/s eta 0:00:02
   --------- ------------------------------ 3.3/14.5 MB 6.7 MB/s eta 0:00:02
   --------- ------------------------------ 3.3/14.5 MB 6.8 MB/s eta 0:00:02
  